In [1]:
import os
import glob
import keras
from keras_video import VideoFrameGenerator
import keras_video.utils
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,MaxPooling2D)
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from collections import deque
from keras.regularizers import l2
from keras.layers import BatchNormalization
from keras.utils.vis_utils import plot_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import os.path
import time
import cv2
import numpy as np

Using TensorFlow backend.


# Loading the Saved Model & Weights

In [2]:
# Set the path to the saved location of the JSON file. 

with open('F:/DLProj/v4_1/train/Model/v1.json', 'r') as json_file:
    json_savedModel= json_file.read()
#load the model architecture 
model = keras.models.model_from_json(json_savedModel)
# Load the saved model weight
model.load_weights('F:/DLProj/v4_1/train/Weight/v1_41.hdf5')

# Testing the model on new data

In [3]:
import numpy as np
img_rows, img_cols, img_depth = 300, 300, 15

In [4]:
def predict_door(input_file, output_file):
    x_train = [] #Variable for storing the files 
    ## Code for extracting 15 frames (distanced at equal interval in the video) from Door Training set
    t_hold = 0.501 #threshold for class prediction
    vid_p = input_file
    interval_duration = 7 # This defines the seconds after which we make a prediction
    frames = []
    legend = []
    cap = cv2.VideoCapture(vid_p)
    fps = cap.get(5) # Getting the frames per second. 
    total_frames = int(cap.get(7))  #Total frames in the video
    vid_dur = int(total_frames/fps) # Gives the video duration in seconds
    intervals = int(vid_dur/interval_duration) #This defines how many predictions we'll make
    if intervals == 0:
        intervals = 1
    frame_per_interval = total_frames //intervals #This defines how many frames are in total for each prediction duration (7 secs)
    frame_distance = frame_per_interval//img_depth #This defines the no. of frames we'll skip.

    frame_count = 0 # This variable keeps count of the total frames, and is reset after extracting each frame
    save_frame = 0 
    interval_count = 1

    for i in range (total_frames): #iterating over all the frames of the video.
            ret, frame = cap.read()
            if ret ==False:  # Checking if the frame is missing, and if missing, we take the previous frame
                frame = f_prev
            elif ret==True:
                f_prev =frame
            frame_count +=1
            if frame_count ==(frame_distance):
                frame_count = 0
                frame=cv2.resize(frame,(img_rows,img_cols),interpolation=cv2.INTER_AREA)
                frames.append(frame)

            if i == ((interval_count*frame_per_interval)-1):
                inpt = np.array(frames)
                inpt = inpt.astype('float32')
                inpt -= np.mean(inpt)
                inpt /=np.max(inpt)
                
                frames = []
                mod_in = np.zeros((1, 15, 300, 300, 3))
                mod_in[0][:][:][:][:] = inpt[:15][:][:][:]
                p = model.predict(mod_in, batch_size = 1)#verbose = 1)
                prob = p[:][0][0]
                s0 = (interval_count-1)*interval_duration #Starting time of the interval in secs
                s1 = (interval_count)*interval_duration   # Ending time of the interval in secs
                print(p)
                if prob >= t_hold:
                    s2 = ['Door: ' +  str(s0) + 'secs - ' + str(s1) + 'secs.']
                else:
                    s2 = ['Not_Door: ' +  str(s0) + 'secs - ' + str(s1) + 'secs.']
                interval_count += 1 #Incrementing interval count by 1
                legend.append(s2)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()
    print(legend)
    with open(output_file, 'w') as f:
        f.write(str(legend))

In [5]:
in_1 = 'F:/Testing/00NN7.MP4'
out_2 = 'F:/Testing/00NN7.txt'

predict_door(in_1 , out_2)

[[0.7573096  0.24269037]]
[[0.7421755  0.25782445]]
[[0.2747539 0.7252461]]
[[0.86289823 0.13710177]]
[['Door: 0secs - 7secs.'], ['Door: 7secs - 14secs.'], ['Not_Door: 14secs - 21secs.'], ['Door: 21secs - 28secs.']]
